# Detect video frames

In [7]:
import cv2, time, pandas
from datetime import datetime

video = cv2.VideoCapture(0)

initial_frame = None
frameCount = 1
times = []
prevStatus = 0
currentStatus = 0

while True:
    frameCount = frameCount + 1
    check, frame = video.read()
    
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gaussian_img = cv2.GaussianBlur(gray_img, (21,21), 0)      
    
    if(initial_frame is None):
        initial_frame = gaussian_img # save initial image
        continue
        
    # capture the difference between initial frame and subsequent frames
    frame_diff = cv2.absdiff(initial_frame, gaussian_img)
    
    # provides threshold such that pixel difference value with less than 30 will be turned black
    # and if the pixel difference value is > 30 then those pixels will be turned white.
    thresh_delta = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta, None, iterations=0)
    
    # defines the contour or the borders for the objects
    (cnts,_) = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if(cv2.contourArea(contour) < 1000): # remove noises and shadows
            continue
        # save current status
        currentStatus = 1
        # find object rectangle areas
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
    
    # This means object appeared infront of camera
    if ((prevStatus == 0) and (currentStatus == 1)): 
        times.append(datetime.now())
        prevStatus = currentStatus

        
    # This means object disappeared
    if ((prevStatus == 1) and (currentStatus == 0)):
        times.append(datetime.now())
        prevStatus = currentStatus
        
    currentStatus = 0
        
    cv2.imshow("Frame", frame)
    cv2.imshow("Gaussian", gaussian_img)
    cv2.imshow("Delta", frame_diff)
    cv2.imshow("Threshold", thresh_delta)
        
    key = cv2.waitKey(1)
    
    if(key == ord('q')):
        break

print(len(times))

# Now use pandas to generate CSV file with the duration of object appearing infront of the camera
df = pandas.DataFrame(columns=["StartTime", "EndTime"])
for i in range(0, len(times), 2):
    df = df.append({ "StartTime": times[i], "EndTime": times[i+1] }, ignore_index=True)
    
df.to_csv("times.csv")

# cv2.waitKey(0)

# print(frameCount)

video.release()

cv2.destroyAllWindows()

8
